In [16]:
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

# 1.准备数据 参数1：文件路径 参数2：oneHot 数组内有一个内容为1则其他全为0
mnist = input_data.read_data_sets('MNIST_data',one_hot=True)
imageInput = tf.placeholder(tf.float32,[None,784])
labelInput = tf.placeholder(tf.float32,[None,10])
trainSize = 500
testSize = 50
trainTimes = 110
learnRate = 0.01
# 2.卷积运算
# 维度转换 n*784->m*28*28*1 2dim->4dim
imageInputReshape = tf.reshape(imageInput,[-1,28,28,1])
# 进行卷积运算：权重w0与偏移b
# 实现正态分布的权重矩阵 参数1：维度(卷积内核：5*5 输出：32 输入;1) 参数2：方差
w0 = tf.Variable(tf.truncated_normal([5,5,1,32],stddev = 0.1))
b0 = tf.Variable(tf.constant(0.1,shape=[32]))
# layer1:激励函数+卷积运算
layer1 = tf.nn.relu(tf.nn.conv2d(imageInputReshape,w0,strides=(1,1,1,1),padding='SAME')+b0)
# 池化层 降维：m*28*28*32 / 1*4*4*1 = m*7*7*32
layer1_pool = tf.nn.max_pool(layer1,ksize=[1,4,4,1],strides=[1,4,4,1],padding='SAME')
# layer2 out:softmax(激励函数+乘加运算) softmax一种回归计算 
w1 = tf.Variable(tf.truncated_normal([7*7*32,1024],stddev=0.1))
b1 = tf.Variable(tf.constant(0.1,shape=[1024]))
# n*7*7*32 -> n*1568(7*7*32)
h_reshape = tf.reshape(layer1_pool,[-1,7*7*32])
# n*1568(7*7*32) x 1568(7*7*32)*1024 = n*1024
h1 = tf.nn.relu(tf.matmul(h_reshape,w1)+b1)
# softmax(n*1024 x 1024*10 = n*10)表示n张图片在0-9即这10个数字中分布的概率
w2 = tf.Variable(tf.truncated_normal([1024,10],stddev=0.1))
b2 = tf.Variable(tf.constant(0.1,shape=[10]))
predict = tf.nn.softmax(tf.matmul(h1,w2)+b2)
# 获得正确结果出现的概率 n*10的矩阵 此处的乘法不是矩阵乘法而是普通乘法
pRight = labelInput*tf.log(predict)
pRightSum = 0
# 求每次测试100份数据的总正确概率
for m in range(0,testSzie):
    for n in range(0,10):
        pRightSum = pRightSum + pRight[m,n]
# 求均值。并通过取反使得满足概率越大误差越小的前提，然后采用梯度下降法
loss = -1*pRightSum/testSize   
# 训练
train = tf.train.GradientDescentOptimizer(learnRate).minimize(loss)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(trainTimes):
        # 每次读取500张进行训练
        images,labels = mnist.train.next_batch(trainSize)
        sess.run(train,feed_dict={imageInput:images,labelInput:labels})
        testImages,testLabels = mnist.test.next_batch(testSize)
        predict_test = sess.run(predict,feed_dict={imageInput:testImages,labelInput:labels})
        # 观察预测结果是是否等于真值
        equal = tf.equal(tf.arg_max(predict_test,1),tf.arg_max(testLabels,1))
        # 计算均值
        equal_float = tf.reduce_mean(tf.cast(equal,tf.float32))
        equal_reslut = sess.run(equal_float,feed_dict={imageInput:testImages,labelInput:testLabels})
        print("第",i,"次",equal_reslut)

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
第 0 次 0.08
第 1 次 0.1
第 2 次 0.14
第 3 次 0.08
第 4 次 0.22
第 5 次 0.24
第 6 次 0.28
第 7 次 0.24
第 8 次 0.26
第 9 次 0.24
第 10 次 0.2
第 11 次 0.28
第 12 次 0.28
第 13 次 0.38
第 14 次 0.28
第 15 次 0.38
第 16 次 0.42
第 17 次 0.44
第 18 次 0.52
第 19 次 0.34
第 20 次 0.24
第 21 次 0.54
第 22 次 0.54
第 23 次 0.42
第 24 次 0.54
第 25 次 0.48
第 26 次 0.56
第 27 次 0.68
第 28 次 0.52
第 29 次 0.68
第 30 次 0.66
第 31 次 0.74
第 32 次 0.58
第 33 次 0.6
第 34 次 0.64
第 35 次 0.6
第 36 次 0.7
第 37 次 0.62
第 38 次 0.64
第 39 次 0.78
第 40 次 0.6
第 41 次 0.58
第 42 次 0.7
第 43 次 0.82
第 44 次 0.78
第 45 次 0.76
第 46 次 0.76
第 47 次 0.64
第 48 次 0.76
第 49 次 0.62
第 50 次 0.78
第 51 次 0.66
第 52 次 0.78
第 53 次 0.62
第 54 次 0.74
第 55 次 0.82
第 56 次 0.8
第 57 次 0.64
第 58 次 0.72
第 59 次 0.7
第 60 次 0.78
第 61 次 0.68
第 62 次 0.76
第 63 次 0.82
第 64 次 0.84
第 65 次 0.78
第 66 次 0.86
第 67 次 0.84
第 68 次 0.